# Hopf Bifurcations/Phonation Onset

In [ ]:
from tqdm.notebook import tqdm
from os.path import isfile, splitext
import itertools
from pprint import pprint
from IPython.core.debugger import set_trace

import h5py
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import dolfin as dfn

from femvf import load, statefile as sf, meshutils
from blockarray import h5utils as bh5utils, blockvec as bv
from vfsig import modal
from vfsig.misc import resample_over_uniform_period

import libhopf
import libsetup
import libsignal
import h5utils
from postprocutils import postprocess

from libsetup import load_transient_model, load_hopf_model

In [ ]:
from main_opt_onsetpressure import (
    ExpParamBasic, ExpParamFreqPenalty, 
    get_dyna_model
)
from exputils import iter_parameters

In [ ]:
## Set default parameters

EMODS = np.arange(2.5, 20, 2.5) * 1e3 * 10
default_params_basic_dict = {
    'MeshName': 'M5_CB_GA3',
    'Ecov': 2.5e4,
    'Ebod': 2.5e4,
    'Functional': 'OnsetPressure'
} 
default_params_penalty_dict = {
    'MeshName': 'M5_CB_GA3',
    'Ecov': 2.5e4,
    'Ebod': 2.5e4,
    'Functional': {
        'Name': 'OnsetPressure',
        'omega': -1,
        'beta': 1000
    }
} 

DEFAULT_PARAMS_BASIC = ExpParamBasic(default_params_basic_dict)
DEFAULT_PARAMS_PENALTY = ExpParamFreqPenalty(default_params_penalty_dict)

def load_sensitivity_vectors(
        substitute_params,
        group_name, 
        default_params=DEFAULT_PARAMS_BASIC
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    props = []
    for params in iter_parameters(substitute_params, default_params):
        fpath = f'out/sensitivity/{params.to_str()}.h5'
        with h5py.File(fpath, mode='r') as f:
            props.append(bh5utils.read_block_vector_from_group(f[group_name]))
    return props

def load_minimization_vectors(
        substitute_params,
        group_name, 
        it=-1,
        default_params=DEFAULT_PARAMS_PENALTY
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    props = []
    for params in iter_parameters(substitute_params, default_params):
        fpath = f'out/minimization/{params.to_str()}.h5'
        with h5py.File(fpath, mode='r') as f:
            props.append(bh5utils.read_block_vector_from_group(f[group_name], nvec=it))
    return props

HOPF_MODEL, *_ = get_dyna_model(DEFAULT_PARAMS_BASIC)
MESH = HOPF_MODEL.res.solid.forms['mesh.mesh']
DG0_DOFMAP = HOPF_MODEL.res.solid.forms['fspace.scalar_dg0'].dofmap()
CELL_TO_SDOF = DG0_DOFMAP.entity_dofs(MESH, 2)

In [ ]:
def plot_tripcolor_sequence(fig, axs, coords, cells, zs):
    zmin = np.min([np.min(z) for z in zs])
    zmax = np.max([np.max(z) for z in zs])

    artists = [
        ax.tripcolor(*coords.T, cells, z, vmin=zmin, vmax=zmax)
        for ax, z in zip(axs, zs)
    ]
    return artists

## Sensitivity


In [ ]:
emods = EMODS

functional_name = 'OnsetFrequency'
# functional_name = 'OnsetPressure'
# functional_name = 'OnsetPressureStrainEnergy'
props_list = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': functional_name}, 
        'props'
    )[0] 
    for emod in emods
]
dprops_list = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': functional_name}, 
        'dprops'
    )[0] 
    for emod in emods
]

fig = plt.figure(figsize=(15, 5))
gs = mpl.gridspec.GridSpec(2, len(emods), figure=fig, height_ratios=[0.05, 0.95])
axs = np.array([fig.add_subplot(gs[-1, i]) for i in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, 0])
print(axs)
# axs = np.atleast_1d(axs)

coords = MESH.coordinates()
cells = MESH.cells()
zs = [dprops['emod'][CELL_TO_SDOF] for dprops in dprops_list]
artists = plot_tripcolor_sequence(fig, axs, coords, cells, zs)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(r"$\frac{dg}{d\mathbf{E}}$")

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
axs.flat[0].set_ylabel("y [cm]")

for ax in axs[1:]:
    ax.tick_params('y', labelleft=False)

fig.tight_layout()


## Minimization


In [ ]:
emods = EMODS[:1]

# functional_name = 'OnsetFrequency'
functional_name = 'OnsetPressure'
# functional_name = 'OnsetPressureStrainEnergy'

props_list = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional/Name': functional_name}, 
        'parameters'
    )[0] 
    for emod in emods
]
dprops_list = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional/Name': functional_name}, 
        'grad'
    )[0] 
    for emod in emods
]

fig = plt.figure(figsize=(15, 5))
gs = mpl.gridspec.GridSpec(2, len(emods), figure=fig, height_ratios=[0.05, 0.95])
axs = np.array([fig.add_subplot(gs[-1, i]) for i in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, 0])

coords = MESH.coordinates()
cells = MESH.cells()
zs = [dprops['emod'][CELL_TO_SDOF] for dprops in dprops_list]
artists = plot_tripcolor_sequence(fig, axs, coords, cells, zs)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(r"$\frac{dg}{d\mathbf{E}}$")

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
axs.flat[0].set_ylabel("y [cm]")

for ax in axs[1:]:
    ax.tick_params('y', labelleft=False)

fig.tight_layout()

In [ ]:
## Global vars
ZETA = 1e-4
R_SEP = 1.0
Y_GAP = 1e-2
# PSUBS = np.concatenate([np.arange(200, 300, 10), np.arange(300, 1000, 100)])*10
PSUBS = np.arange(550, 650, 10) * 10 
ECOV = 5e3 * 10
# EBODY = 15e3 * 10
EBODY = 5e3 * 10

# Load the transient model
MESH_NAME = 'BC-dcov5.00e-02-cl1.00'
MESH_NAME = 'M5_CB_GA3'

mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

# print(model_tran.solid.forms['mesh.vertex_label_to_id'])
# print(model_tran.solid.forms['mesh.facet_label_to_id'])
# print(model_tran.solid.forms['mesh.cell_label_to_id'])

In [ ]:
## Initialize functions to extract glottal width from saved simulations

# Number of points per period to use in processing
N_PER_PERIOD = 100
proc_gw_tran = make_sig_glottal_width_sharp(model_tran)
def time(f):
    return np.array(f.get_times())

In [ ]:
## Load mesh and DOF map information for plotting data on the mesh
TRI = res.solid.forms['mesh.mesh'].coordinates()
X, Y = TRI[:, 0], TRI[:, 1]
CELLS = res.solid.forms['mesh.mesh'].cells()

# These indices select DOFs from a scalar function in vertex-order
IDX_VERT = dfn.vertex_to_dof_map(res.solid.forms['fspace.scalar'])

XREF = res.solid.XREF.vector()
VERT_TO_VDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])

# Conversion for centimeter
CM = 1/2.54

## Post-process nonlinear oscillations (transient simulations)

In [ ]:
## Post processing transient simulations
OUT_DIR = f'out/zeta{ZETA:.2e}_rsep{R_SEP:.1f}_ygap{Y_GAP:.2e}_init{INIT_STATE_TYPE}_fixed_rsep'
postprocess_fname = f'{OUT_DIR}/data.h5'
signal_to_proc = {
    'glottal_width': proc_gw_tran,
    'time': time}
files = [f'{OUT_DIR}/{case_config(MESH_NAME, psub, ECOV, EBODY)}.h5' for psub in PSUBS]

# `SIGNALS` is a dictionary mapping each `case` and signal name to a time signal
SIGNALS = postprocess(postprocess_fname, files, model_tran, signal_to_proc)

# with h5py.File(postprocess_fname, mode='r') as f:
#     SIGNALS = h5utils.h5_to_dict(f, {})

In [ ]:
## Resample the transient simulations so they have a fixed number of points per period
cases = list(set([key.split('/')[0] for key in SIGNALS.keys()]))

# NOFFSET controls the portion of the signal to treat as 'steady state'
# -8124, sets the final 8124 samples to be considered a steady state portion
# This portion of the signal is resampled on a periodic basis
NOFFSET = -2**10
NPERIOD = 5


MEASURES = {}
for case in cases:
    # F0 and period are in units of samples
    fund_freq, _, df = modal.estimate_fundamental_mode(
        SIGNALS[f'{case}/glottal_width'][NOFFSET:]
    )
    MEASURES[f'{case}/fund_freq'] = fund_freq
    MEASURES[f'{case}/dfreq'] = df
    
    MEASURES[f'{case}/period'] = fund_freq**-1

RESIGNALS = {}
for case in cases:
    xp = np.arange(SIGNALS[f'{case}/glottal_width'][NOFFSET:].size)
    fp = SIGNALS[f'{case}/glottal_width'][NOFFSET:]
    
    # `num_period` is the number of periods/cycles in the sequence
    # This should always be an integer due to how FFT computes the
    # signal frequencies
    samples_per_period = MEASURES[f'{case}/period']
    num_period = int(round(fp.size/samples_per_period))
    
    x = np.linspace(0, num_period, int(num_period)*N_PER_PERIOD + 1) * samples_per_period
    
    RESIGNALS[f'{case}/glottal_width'] = np.interp(x, xp, fp)

resignals_update = {
    f'{case}/time': 
    np.arange(RESIGNALS[f'{case}/glottal_width'].size)/MEASURES[f'{case}/period']
    for case in cases}

RESIGNALS.update(resignals_update)

In [ ]:
## Post processing Hopf simulation (load the Hopf bifurcation state)
hopf_sim_fpath = f"out/hopf_state.h5"

with h5py.File(hopf_sim_fpath, mode='r') as f:
    xhopf = bh5utils.read_block_vector_from_group(f)

# Decompose the Hopf state vector to common components
xfp_hopf = xhopf[:4]
mode_hopf_real, mode_hopf_imag = xhopf[4:8], xhopf[8:12]
# mode_hopf = mode_hopf_real + 1j*mode_hopf_imag
omega_hopf = xhopf['omega'][0]
psub_hopf = xhopf['psub'][0]

unit_mode_real, unit_mode_imag = libhopf.normalize_eigenvector_amplitude(
    mode_hopf_real, mode_hopf_imag)

## Plots

### Nonlinear oscillations

In [ ]:
## Plot transient simulations
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for psub in PSUBS:
    case = case_config(MESH_NAME, psub, ECOV, EBODY)
    ret = RESIGNALS[f'{case}/time']
    regw = RESIGNALS[f'{case}/glottal_width']
    
    t = SIGNALS[f'{case}/time']
    dt = t[1] - t[0]
    freq = MEASURES[f'{case}/fund_freq'] / dt
    dfreq = MEASURES[f'{case}/dfreq'] / dt
    
    # t = SIGNALS[f'{case}/time']
    # gw = SIGNALS[f'{case}/glottal_width']
    
    ax.plot(ret, regw, label=f"$p_{{sub}}$: {psub/10:.1f} Pa $F_0$: {freq:.1f} +- {dfreq:.1f} Hz", lw=1.0)
    
# ax.set_xlim(60, 70)
# ax.set_xlim(0, 2)

ax.set_xlabel("Time [period]")
ax.set_ylabel("Glottal width [cm]")
ax.legend(loc='lower left', bbox_to_anchor=(0, 1))

fig.tight_layout()
fig.savefig(f'fig/transient_glottal_width.png')

### Small amplitude oscillations

In [ ]:
## Plot Hopf simulations
fig, ax = plt.subplots(1, 1)

for ampl in np.linspace(0, 20000.0, 5):
    gw = proc_gw_hopf(
        xhopf.to_mono_ndarray(),
        np.array([ampl, 0.0]))
    ax.plot(gw, label=f"Amplitude {ampl:.2e}")
    
ax.set_xlabel(f"Time [period]")
ax.set_ylabel("Glottal width [cm]")
ax.legend()

fig.tight_layout()
fig.savefig("fig/onset_gw_vs_amplitude.png", dpi=250)

In [ ]:
## Plot the onset mode shape
import dolfin as dfn
XREF = res.solid.XREF.vector()

NPHASE = 5
phases = 2*np.pi*np.linspace(0, 1, NPHASE+1)[:-1]

VERT_TO_VECDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])
CELLS = res.solid.forms['mesh.mesh'].cells()

fig, axs = plt.subplots(1, NPHASE, sharey=True, sharex=True, figsize=(10, 5))
for ax, phase in zip(axs, phases):
    ampl = 5e4*np.exp(1j*phase)
    mode_hopf_dof_order = (
        XREF[:] + xhopf['u'] 
        + np.real(
            ampl * (
                xhopf['u_mode_real'] 
                + 1j*xhopf['u_mode_imag']
            )
        )
    )
    
    mode_hopf_vert_order = mode_hopf_dof_order[VERT_TO_VECDOF].reshape(-1, 2)
    # print(mode_hopf_vert_order.shape)
    ax.triplot(*mode_hopf_vert_order.T, triangles=CELLS)
    ax.set_title(f"${phase/np.pi:.2f}\pi$ rad")
    
for ax in axs:
    ax.set_aspect('equal')
    ax.set_xlabel("x [cm]")
axs[0].set_ylabel("y [cm]")
    
fig.tight_layout()
fig.savefig(f'fig/hopf_mode_shape.png', dpi=200)

In [ ]:
def plot_onset_mode(axs):

### Compare nonlinear and small-amplitude oscillations

In [ ]:
## Plot Transient vs Hopf simulations
SCALE = np.array([1e4, 1])

def sqr_err(sc_ampl_phase, gw_ref):
    """
    Return the squared error between small amplitude and reference glottal widths
    """
    ampl_phase = sc_ampl_phase*SCALE
    gw_hopf = proc_gw_hopf(
        xhopf.to_mono_ndarray(), 
        ampl_phase)
    
    # Weight the reference glottal width measurements
    # linearly with glottal width; larger glottal widths
    # get higher weights
    weight = gw_ref/jnp.abs(gw_ref.max()) 
    
    err = weight*(gw_hopf-gw_ref)**2
    return jnp.sum(err)

from scipy import optimize

fig, ax = plt.subplots(1, 1)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
psubs = np.array([600, 700, 800, 900])*10
psubs = PSUBS
# psubs = PSUBS[0::2]

cases = [case_config(MESH_NAME, psub, ECOV, EBODY) for psub in psubs]
for ii, (psub, color, case) in enumerate(zip(psubs, colors, cases)):
    # For each case, load the transient simulation glottal width 
    t = RESIGNALS[f'{case}/time'][-N_PER_PERIOD:]
    gw_tran = RESIGNALS[f'{case}/glottal_width'][-N_PER_PERIOD:]
    
    # Calculate an optimal [phase, amplitude] for the onset glottal width
    # to minimize the difference from the onset glottal width
    ampl_phase0 = np.array([5e3, 0.0])
    opt_res = optimize.minimize(sqr_err, ampl_phase0/SCALE, args=(gw_tran,), jac=jax.grad(sqr_err, 0), method='BFGS')
    ampl_phase = opt_res['x'] * SCALE
    # print(opt_res)
    
    # Compute the onset glottal width with the optimal phase and amplitude
    gw_hopf0 = proc_gw_hopf(
        xhopf.to_mono_ndarray(),
        ampl_phase0)
    
    gw_hopf = proc_gw_hopf(
        xhopf.to_mono_ndarray(),
        ampl_phase)
    
    ax.plot(t, gw_tran, color=color, ls='-')
    ax.plot(t, gw_hopf, color=color, ls='-.')
    ax.plot(t, gw_hopf0, color=color, ls=':')
    
ax.set_xlabel("t/T")
ax.set_ylabel("Glottal width [cm]")

from matplotlib.lines import Line2D

f0s = []
for case in cases:
    _t = SIGNALS[f'{case}/time']
    dt = _t[1]-_t[0]
    f0s.append((MEASURES[f'{case}/period'] * dt)**-1)

# Make a legend describing the relationship 
# ls -> model type
# psub -> color
_model_type_ls = ['-', '-.']
_model_type_lines = [Line2D([0], [0], color='k', ls=ls) for ls in _model_type_ls] 
_model_type_labels = ["Transient", f"Onset {omega_hopf/2/np.pi:.1f} Hz {psub_hopf/10:.1f} Pa"]

_psub_colors = colors[:len(psubs)]
_psub_lines = [Line2D([0], [0], color=color) for color in _psub_colors]
_psub_labels = [f"{psub/10:.0f} Pa, {f0:.1f} Hz" for psub, f0 in zip(psubs, f0s)]
ax.legend(
    _model_type_lines+_psub_lines,
    _model_type_labels+_psub_labels,
    loc=(0, 1)
)

fig.savefig('fig/transient_vs_onset_gw.png', dpi=200)

### Optimization results

#### Plotting functions

In [ ]:
def plot_opt_summary(fig, its, objs, grads_norm, it_descr):
    """
    Creates a gridded layout figure summarizing the optimization history
    
    Parameters
    ----------
    objs : objective function history
    grads_norm : gradient norm history
    it_descr :
        tuple of (n, emod, demod) specifying a specific iteration,
        and the modulus and modulus gradient at that iteration.
    """
    # fig = plt.figure(figsize=(6, 6))
    gs = gridspec.GridSpec(3, 2, figure=fig, height_ratios=[1, 1, 2])

    # Plot the objective function/gradient norm history
    ax_obj = fig.add_subplot(gs[0, :])
    ax_grad = fig.add_subplot(gs[1, :])
    ax_obj.sharex(ax_grad)
    ax_obj.xaxis.set_tick_params(labelbottom=False)

    ax = ax_obj
    ax.plot(its, objs)
    ax.axvline([its[N]], color='k', ls='-.', alpha=0.5)

    ax.set_ylabel("Functional")

    ax.set_ylim([0.9*np.nanmin(objs), 1.1*objs[0]])
    ax.set_yscale('log')

    ax = ax_grad
    ax.plot(its, grads_norm)
    ax.axvline([its[N]], color='k', ls='-.', alpha=0.5)
    # ax.set_ylim([0.8*grads_norm[0], 1.2*grads_norm[0]])
    ax.set_xlim(its[[0, -1]])
    ax.set_ylabel("||grad||")
    ax.set_yscale('log')
    
    ax.set_xlabel("Iteration")

    # Plot the current parameter guess
    ax_mod = fig.add_subplot(gs[2, :])
    ax = ax_mod

    mappable = ax.tripcolor(X, Y, CELLS, param['emod'][IDX_VERT]/10/1e3)
    ax.set_xlabel("x [cm]")
    ax.set_ylabel("y [cm]")

    cbar = fig.colorbar(mappable, ax=ax)
    cbar.ax.set_ylabel("$E$ [kPa]")
    ax.set_aspect(1)

    ## Plot the current gradient
    # ax_grad = fig.add_subplot(gs[2, 1])
    # ax = ax_grad
    # mappable = ax.tripcolor(X, Y, CELLS, grad['emod'][IDX_VERT]/10/1e3)
    # ax.set_xlabel("x [cm]")
    # ax.set_ylabel("y [cm]")
    # cbar = fig.colorbar(mappable, ax=ax)
    # cbar.ax.set_ylabel("$dE$ [Pa/kPa]")
    # ax.set_aspect(1)
    return fig

def plot_onset_hist(fig, its, onset_ps, onset_fs):
    """
    Creates a figure summarizing the onset pressure and frequency history
    
    Parameters
    ----------
    objs : objective function history
    grads_norm : gradient norm history
    it_descr :
        tuple of (n, emod, demod) specifying a specific iteration,
        and the modulus and modulus gradient at that iteration.
    """
    gs = gridspec.GridSpec(2, 1, figure=fig)

    # Plot the objective function history
    ax_onp = fig.add_subplot(gs[0, :])
    ax_onf = fig.add_subplot(gs[1, :])
    ax_onp.xaxis.set_tick_params(labelbottom=False)

    ax_onp.plot(its, np.array(onset_ps)/10)
    ax_onf.plot(its, np.array(onset_fs)/2/np.pi)

    ax_onp.set_ylabel("Onset pressure [Pa]")
    ax_onf.set_ylabel("Onset frequency [Hz]")
    ax_onf.set_xlabel("Iteration")
    return fig

def plot_onset_mode(fig, axs, xhopf, phases=None, ampl=1.0):
    """
    Plot the sequence of deformations of the onset mode
    
    Parameters
    ----------
    onset_mode :
        A block vector representing the onset Hopf system solution
    ampl :
        A complex number representing the amplitude of the mode
    """
    NPHASE = len(axs)
    if phases is None:
        phases = np.linspace(0, 2*np.pi, NPHASE+1)[:-1]
    
    xy_ref = XREF[:] + xhopf['u'] 
    for ax, phase in zip(axs, phases):
        dxy = np.real(
            ampl * (
                xhopf['u_mode_real'] 
                + 1j*xhopf['u_mode_imag']
            )
            * np.exp(1j*phase)
        )

        xy_ = (xy_ref+dxy)[VERT_TO_VDOF].reshape(-1, 2)
        ax.triplot(*xy_.T, triangles=CELLS)
    return fig, axs

def compute_bifurcation_trend(res, psubs):
    """
    Compute the trend of the most unstable mode as a function of bifurcation parameter
    
    This can be used to create classic eigenvalue crossing diagrams for the most unstable mode
    """
    # psubs = np.arange(300, 600, 2.5)*10
    # psubs = np.arange(0, 1500, 100)*10
    omegas_max = np.zeros(psubs.shape, dtype=complex)

    xfp_0 = res.state.copy()
    xfp_0.set(0.0)
    for ii, psub in enumerate(psubs):
        res.control['psub'][0] = psub
        res.set_control(res.control)

        xfp_n, info = libhopf.solve_fp_newton(res, xfp_0, psub, newton_params={'max_iter': 20})
        # print(f"Solving for fixed point took {info['num_iter']} iterations. Abs err {info['abs_errs'][-1]}")
        omegas, eigvecs_real, eigvecs_imag = libhopf.solve_modal(res, xfp_n, psub)

        idx_max = np.argmax(omegas.real)
        omegas_max[ii] = omegas[idx_max]
    return omegas_max

def load_opt_hist(f):
    n = f['objective'].shape[0]
    for key in ['grad', 'parameters', 'hopf_state']:
        group = f[key]
        n = min(group[list(group.keys())[0]].shape[0], n)
    
    objs = f['objective'][:n]
    grads = [bh5utils.read_block_vector_from_group(f['grad'], nn) for nn in range(n)]
    params = [bh5utils.read_block_vector_from_group(f['parameters'], nn) for nn in range(n)]
    xhopfs = [bh5utils.read_block_vector_from_group(f['hopf_state'], nn) for nn in range(n)]
    return objs, grads, params, xhopfs


def proc_gw_hopf(xhopf, camp, n):
    _proc_gw_hopf = libsignal.make_glottal_width(res_hopf, num_points=n)
    return _proc_gw_hopf(xhopf, camp)

#### Onset pressure minimization

In [ ]:
## Specify the case and load the minimization history
alpha = 1e-8
# alpha = 0.0 
emod = 5.5e3 * 10
fname = f'opt_hist_emod{emod:.2e}_alpha{alpha:.2e}'
fpath = f'out/minimize_onset_pressure/{fname}.h5'

with h5py.File(fpath, mode='r') as f:
    print(fpath)
    print(f.keys())
    objs, grads, params, xhopfs = load_opt_hist(f)

its = np.arange(len(objs))
grads_norm = np.array([bvec.norm(grad) for grad in grads])
onset_ps = np.array([xhopf['psub'][0] for xhopf in xhopfs])
onset_fs = np.array([xhopf['omega'][0] for xhopf in xhopfs])

In [ ]:
N = np.nanargmin(objs)
# N = 100
param = params[N]
grad = grads[N]
xhopf = xhopfs[N]

fig = plt.figure(figsize=(6, 6))
fig = plot_opt_summary(fig, its, objs, grads_norm, (N, param['emod'][IDX_VERT], grad['emod'][IDX_VERT]))
fig.tight_layout()
fig.savefig(f'fig/OptHist_{fname}.png', dpi=200)

fig = plt.figure(figsize=(6, 4))
fig = plot_onset_hist(fig, its, onset_ps, np.abs(onset_fs))
fig.savefig(f'fig/OnsetHist_{fname}.png', dpi=200)
print(onset_ps[N])

nphase = 10
phases = np.linspace(0, 2*np.pi, nphase+1)[:-1]
fig, axs = plt.subplots(1, 10, sharex=True, figsize=(20, 8))
fig, axs = plot_onset_mode(fig, axs, xhopf, phases, ampl=1e4)
for ax, phase in zip(axs, phases):
    ax.set_adjustable('box')
    ax.set_title(f"${phase/np.pi:.2f}\pi$ rad")
    ax.set_aspect('equal')
    ax.set_xlabel("x [cm]")
axs[0].set_ylabel("y [cm]")
fig.tight_layout()
fig.savefig(f'fig/ModeShape_{fname}.png', dpi=200)

In [ ]:
res_hopf.set_props(param)
res_hopf.set_state(bvec.convert_subtype_to_petsc(xhopf))
omegas, *_eigvecs = libhopf.solve_modal(res_hopf.res, xhopf[res_hopf.labels_fp], onset_ps[N])
print(omegas)

In [ ]:
psubs = np.linspace(onset_ps[N]/10-100, onset_ps[N]/10+500, 81)*10
res_hopf.set_props(param)
omegas_unstable = compute_bifurcation_trend(res_hopf.res, psubs)

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)
axs[0].plot(psubs, omegas_unstable.real)
axs[1].plot(psubs, omegas_unstable.imag)
axs[0].axhline(0, color='k')

for ax in axs:
    ax.axvline(onset_ps[N], color='grey', ls='-.')

axs[0].set_ylabel("$\omega_r$")
axs[1].set_ylabel("$\omega_i$")
axs[1].set_xlabel("$P_{sub}$ [Ba]")
fig.savefig(f'fig/Bifurcation_{fname}.png', dpi=200)

#### Inverse analysis stress test

In [ ]:
def form_opt_fname(ecov_ini, ebod_ini, ecov_gt, ebod_gt, alpha):
    return (
        f'OptInv_emod{ecov_ini:.2e}_ebody{ebod_ini:.2e}'
        f'_alpha_{alpha:.2e}'
        f'_gtLargeAmp_ecov{ecov_gt:.2e}_ebody{ebod_gt:.2e}'
    )

In [ ]:
## Specify cases from the 'stress test'
# value for ini are (2.5+5)/2*1e4, (5+7.5)/2*1e4, ...
emod_inis = np.array([(2.5+5)/2, (5+7.5)/2, (7.5+10)/2, (10+12.5)/2])*1e4
emod_cov_ini = emod_inis[3]
emod_bod_ini = emod_inis[3]
emod_cov_gt = 2.5e4
emod_bod_gt = 10.0e4

# alpha is in {0, 1e-12, 1e-10, 1e-8, 1e-6}
alpha = 1e-10
# alpha = 0.0

opt_fname = form_opt_fname(emod_cov_ini, emod_bod_ini, emod_cov_gt, emod_bod_gt, alpha)
opt_fpath = f'out/stress_test/{opt_fname}.h5'

gt_fname = f'LargeAmp_ecov{emod_cov_gt:.2e}_ebody{emod_bod_gt:.2e}'
gt_fpath = f'out/stress_test/{gt_fname}.h5'
print(gt_fpath)

with h5py.File(opt_fpath, mode='r') as f:
    print(opt_fpath)
    print(f.keys())
    objs, grads, params, xhopfs = load_opt_hist(f)
    
    gw_gt, freq_gt = f['gw_ref'][:], f['omega_ref'][()]
    
its = np.arange(len(objs))
grads_norm = np.array([bvec.norm(grad) for grad in grads])
onset_ps = np.array([xhopf['psub'][0] for xhopf in xhopfs])
onset_fs = np.array([xhopf['omega'][0] for xhopf in xhopfs])

In [ ]:
## Plot a comparison of the estimated glottal width vs measured glottal width
N = np.nanargmin(objs)
# print(N, len(objs))
# N = -1
param = params[N]
grad = grads[N]
xhopf = xhopfs[N]
print(len(xhopfs))
print(len(params))
print(len(grads))
print(len(objs))
print(N)

In [ ]:
std_gw = (0.1/5) / (np.maximum(gw_gt, 0.0) / gw_gt.max())

In [ ]:
campl = param[-2:]
gw_hopf = proc_gw_hopf(
    xhopf.to_mono_ndarray(),
    campl.to_mono_ndarray(),
    gw_gt.size
)
freq_hopf = xhopf['omega'][0]/2/np.pi

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10*CM, 5*CM))

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

gt_color = colors[0]
# ax.plot(t_gt, gw_gt, label=f"Ground truth ({freq_gt:.1f} $\pm$ {dfreq:.1f} Hz)")
# ax.plot(t_gt, gw_gt + std_gw, color=gt_color, ls=':', label="+-stdev")
# ax.plot(t_gt, gw_gt - std_gw, color=gt_color, ls=':')
# ax.plot(t_hopf, gw_hopf, color=colors[1], label=f"Estimated ({freq_hopf:.1f} Hz)")

ax.plot(gw_gt, label=f"Ground truth ({freq_gt:.1f} $\pm$ Hz)")
ax.plot(gw_gt + std_gw, color=gt_color, ls=':', label="+-stdev")
ax.plot(gw_gt - std_gw, color=gt_color, ls=':')
ax.plot(gw_hopf, color=colors[1], label=f"Estimated ({freq_hopf:.1f} Hz)")

ax.set_ylabel("Glottal width [cm]")
ax.set_xlabel("Time [samples]")
# ax.set_xlim(t_hopf[[0, -1]])
ax.set_ylim([
    0.0, 
    1.5*max(gw_hopf.max(), gw_gt.max())
])

ax.legend()
fig.tight_layout()

fig.savefig(f'fig/GWComparison_{opt_fname}.png', dpi=200)
print(objs[N])
std_omega = 10.0
obj_manual = np.sum(1/std_gw*(gw_gt-gw_hopf)**2) + np.sum(1/std_omega*(2*np.pi*(freq_gt-freq_hopf))**2)
print(obj_manual)

In [ ]:
## Plot a summary of the objective function history and parameters at specific iterations
N = its[-1]
param = params[N]
grad = grads[N]

fig = plt.figure(figsize=(10*CM, 10*CM))
fig = plot_opt_summary(fig, its, objs, grads_norm, (N, param['emod'][IDX_VERT], grad['emod'][IDX_VERT]))
fig.tight_layout()

fig.savefig(f'fig/OptHist_{opt_fname}.png', dpi=200)

#### Onset conditions vs layer moduli


In [ ]:
_emod = np.arange(2.5, 12.5+2.5, 2.5)*1e3*10
x = _emod
y = _emod

ps_onset = np.zeros((x.size, y.size))
fs_onset = np.zeros((x.size, y.size))

for (ii, ecov), (jj, ebod) in itertools.product(enumerate(x), enumerate(y)):
    _fname = f'Hopf_ecov{ecov:.2e}_ebody{ebod:.2e}'
    _fpath = f'out/stress_test/{_fname}.h5'
    
    ponset = np.nan
    fonset = np.nan
    if isfile(_fpath):
        with h5py.File(_fpath, mode='r') as f:
            if 'state' in f:
                xhopf = bh5utils.read_block_vector_from_group(f['state'])
                ponset = xhopf['psub'][0]
                fonset = xhopf['omega'][0]
    
    ps_onset[ii, jj] = ponset
    fs_onset[ii, jj] = fonset

In [ ]:
print(np.nanmax(ps_onset))

In [ ]:
fig, axs = plt.subplots(2, 1, sharey=True, sharex=True)

mappable = axs[0].contourf(x/10/1e3, y/10/1e3, ps_onset/10/1e3)
cbar = fig.colorbar(mappable, ax=axs[0])
cbar.ax.set_ylabel("$P_{onset}$ [kPa]")

mappable = axs[1].contourf(x/10/1e3, y/10/1e3, fs_onset/2/np.pi)
cbar = fig.colorbar(mappable, ax=axs[1])
cbar.ax.set_ylabel("$\omega_{onset}$ [Hz]")

for ax in axs:
    ax.set_ylabel("$E_{cover}$ [kPa]")
axs[-1].set_xlabel("$E_{body}$ [kPa]")

for ax in axs:
    ax.set_aspect(1)

### Generic bifurcation plots

Plots of the growth rate vs subglottal pressure.

In [ ]:
MESH_NAME = 'M5_CB_GA1'

mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

In [ ]:
for key, subvec in res.props.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")
    
for key, subvec in res.control.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")

In [ ]:
psubs = np.arange(0, 800, 20)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 5), sharex=True)

axs[0].plot(psubs, modes.real)
axs[1].plot(psubs, modes.imag)

axs[0].axhline(0, color='k', ls='-.')

axs[0].set_ylabel("$\omega_{real}$")
axs[1].set_ylabel("$\omega_{imag}$")
axs[1].set_xlabel("$p_{sub}$")

# axs[0].legend()
# axs[1].legend()

fig.tight_layout()

### Bifurcation plots for smooth minimum separation fluid models

In [ ]:
# Set the model properties and sanity check the values
_region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar']
)
props = libsetup.set_props(res.props, _region_to_dofs, res)

props['zeta_min'] = 1e-8
props['zeta_sep'] = 1e-4
res.set_props(props)

# proplabel_to_norm = {label: subvec.norm() for label, subvec in props.items()}
# pprint(proplabel_to_norm)

proplabel_to_max = {label: subvec.max() for label, subvec in props.items()}
pprint(proplabel_to_max)
print(res.solid.forms['mesh.mesh'].coordinates()[..., 1].max())

#### Plot the growth rate vs $p_{sub}$

In [ ]:
zeta_min = 1e-6
zeta_sep = 1e-2
props = res.props
props['zeta_min'] = zeta_min
props['zeta_sep'] = zeta_sep
res.set_props(props)

# psubs = np.arange(585, 590, 0.1)*10
psubs = np.arange(700, 800, 10)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
def jac_block_norms(res, psub):
    xfp, info = libhopf.solve_fp(res, psub)
    res.set_state(xfp)
    res.control['psub'].array[0] = psub
    res.set_control(res.control)
    
    dres_dstate = res.assem_dres_dstate()
    return tuple([submat.norm() for submat in dres_dstate.subarrays_flat])

block_norms = [jac_block_norms(res, psub) for psub in tqdm(psubs)]
block_labels = [','.join(multi_label) for multi_label in itertools.product(res.state.labels[0], res.state.labels[0])]

block_norms = np.array(block_norms)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(5, 5), sharex=True)
    
ii_max = np.argmax(modes.real)
print(psubs[ii_max])

axs[0].plot(psubs, modes.real)
axs[0].axhline(0, color='k', ls='-.')
axs[0].set_ylabel("$\omega_{real}$")

axs[1].plot(psubs, modes.imag)
axs[1].set_ylabel("$\omega_{imag}$")

for norms, label in zip(block_norms.T, block_labels):
    axs[2].plot(psubs, norms, label=label)
axs[2].set_ylabel("$|| \\frac{dF}{dx} ||$")
axs[2].set_xlabel("$p_{sub}$")
# axs[2].set_yscale('log')
axs[2].legend()

# axs[0].set_ylim(-10, 50)

fig.savefig(f'fig/GrowthRatevsPsub_zetamin{zeta_min:.2e}_zetasep{zeta_sep:.2e}_closeup.png', dpi=200)

### Strange bifurcation behaviour for smoothed minimum separation models

Rapid changes in eigenvalues can occur with small changes in $p_{sub}$ when using a smooth minimum approximation. This happens when the smooth minimum approximation approaches the true minimum where small changes in areas rapidly shift weights between nodes.

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True)

s = np.linspace(0, 1, 12)
a = (s-0.5)**2 + 0.1
da = 1e-2*s

def smoothmin(a, zeta=1e-4):
    log_w = -a/zeta
    log_w = log_w - log_w.max()
    print(log_w)
    _w = np.exp(log_w)
    print(_w.max())
    return _w/np.sum(_w)

axs[0, 0].plot(s, a, marker='.')
axs[0, 0].set_title("a [cm]")
axs[1, 0].plot(s, smoothmin(a))
axs[1, 0].set_ylim(0, 1)

axs[0, 0].set_ylabel("Area [cm]")
axs[1, 0].set_ylabel("smoothmin weight")

axs[0, 1].plot(s, a+da, marker='.')
axs[0, 1].set_title("a + da [cm]")
axs[1, 1].plot(s, smoothmin(a+da))
axs[1, 1].set_ylim(0, 1)

# axs[0, 0].yaxis.
for ax_row in axs:
    for col in range(1, ax_row.size):
        ax_row[0].sharey(ax_row[col])
        ax_row[col].yaxis.set_tick_params(labelleft=False)

## Debugging

For the case where we minimize onset pressure while maintaining a constant onset frequency, the optimization encounters a problem. This is illustrated below for the history file 'OPT_DEBUG.h5' which started from a uniform initial guess of 5 kPa.

At some of the last iterations, optimization progress stalls and the optimization routine fails with a zero size step on a line search. A likely reason why this occurs can be seen by plotting the maximum real eigenvalue as a function of subglottal pressure. The minimal subglottal pressure of 0.38 kPa occurs due to a very sharp peak in the profile. It is likely that as parameters change in the line search, this small peak disappears so that onset jumps to the next onset pressure of around 0.5 kPa.

In [ ]:
# Load optimization history
opt_fname = 'OPT_DEBUG_1'
with h5py.File(f'out/{opt_fname}.h5', mode='r') as f:
    objs, grads, params, xhopfs = load_opt_hist(f)
    
its = np.arange(len(objs))
grads_norm = np.array([bvec.norm(grad) for grad in grads])
onset_ps = np.array([xhopf['psub'][0] for xhopf in xhopfs])
onset_fs = np.array([xhopf['omega'][0] for xhopf in xhopfs])

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True)

axs[0].plot(its, objs)
axs[0].set_ylabel("Objective")

axs[1].plot(its, onset_ps)
axs[1].set_ylabel("Onset pressure")

axs[2].plot(its, np.abs(onset_fs))
axs[2].set_ylabel("Onset freq.")

axs[2].set_xlabel("")

### Compute a line search along a given direction

In [ ]:
# compute functionals along the line from N -> N+1
N = 25
res_hopf.set_state(bvec.convert_subtype_to_petsc(xhopfs[N]))
res_hopf.set_props(bvec.convert_subtype_to_petsc(params[N]))

xhopf_n, info = libhopf.solve_hopf_newton(res_hopf, res_hopf.state)
print(info)
print(xhopf_n['psub'][0])

### Compute the spectrum at a specific iteration

In [ ]:
# Specify the parameter set from the optimization to use
N = 26
res.set_props(bvec.convert_subtype_to_petsc(params[N][:-2]))

In [ ]:
## Plot the spectrum at the onset pressure

def plot_spectrum(fig, ax, xhopf):
    res.set_props(bvec.convert_subtype_to_petsc(params[N][:-2]))
    res.set_control(res.control)

    xfp_n = xhopf[res_hopf.labels_fp]
    psub = xhopf['psub'][0]
    omegas, eigvecs_real, eigvecs_imag = libhopf.solve_modal(res, xfp_n, psub)
    
    ax.scatter(omegas.real, omegas.imag)
    ax.axhline(0, color='k')
    ax.axvline(0, color='k')

    ax.set_xlabel("$\Re(\omega)$")
    ax.set_ylabel("$\Im(\omega)$")
    ax.set_title(f"$P_\mathrm{{onset}}$={onset_ps[N]/10:.1f} Pa")
    ax.set_xlim(-50, 10)
    ax.set_ylim(-1200, 1200)

    fig.tight_layout()
    return fig, ax

# for n in range(0, 27):
#     fig, ax = plt.subplots(1, 1)
#     fig, ax = plot_spectrum(fig, ax, n)
#     fig.savefig(f"fig/Spectrum{n}.png")
#     plt.close(fig)
    
fig, ax = plt.subplots(1, 1)
fig, ax = plot_spectrum(fig, ax, xhopfs[N])

In [ ]:
## Compute the most unstable eigenvalue over a range of subglottal pressures
# This gives a rough idea at which subglottal pressure(s) (if any) onset will occur

# psubs = np.arange(300, 600, 2.5)*10
psubs = np.arange(0, 2600, 100)*10
omegas_max = np.zeros(psubs.shape, dtype=complex)

least_stable_modes_info = [libhopf.solve_least_stable_mode(res_hopf.res, psub) for psub in psubs]
omegas_max = np.array([least_stable_info[0] for least_stable_info in least_stable_modes_info])

# xfp_0 = res.state.copy()
# xfp_0.set(0.0)

# for ii, psub in enumerate(psubs):
#     res.control['psub'][0] = psub
#     res.set_control(res.control)

#     xfp_n, info = libhopf.solve_fixed_point_newton(res, xfp_0, newton_params={'max_iter': 20})
#     # print(f"Solving for fixed point took {info['num_iter']} iterations. Abs err {info['abs_errs'][-1]}")
#     omegas, eigvecs_real, eigvecs_imag = libhopf.solve_linear_stability(res, xfp_n)
    
#     idx_max = np.argmax(omegas.real)
#     omegas_max[ii] = omegas[idx_max]

In [ ]:
print(np.array(omegas_max).shape)
print(psubs.shape)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 4), sharex=True)

# idx = np.logical_and(psubs >= 2000, psubs <= 6000)
idx = np.ones(psubs.shape, dtype=bool)

axs[0].plot(psubs[idx], np.array(omegas_max)[idx].real)
axs[0].axhline(0, color='k')
axs[0].set_ylabel("$real \; \omega$ $[rad/s]$")


axs[1].plot(psubs[idx], np.array(omegas_max)[idx].imag)
axs[1].axhline(0, color='k')
axs[1].set_ylabel("$imag \; \omega$ $[rad/s]$")

# axs[1].set_xlim(3500, 4000)5
# axs[0].set_ylim()


axs[1].set_xlabel("$P_{sub}$ $[Pa]$")